In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
Obj = np.loadtxt("BaseLogs/objectLog.txt", delimiter = "\n")
DC = np.loadtxt("BaseLogs/DCLog.txt", delimiter = "\n")

In [33]:
X = pd.read_csv('BaseLogs/XLog.txt', sep = ' ', header = None)
#print(X.head())
print(X.shape)
X.tail()

(271, 480)


,0,1,2,3,4,5,6,7,8,9,...,470,471,472,473,474,475,476,477,478,479
266,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
267,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
268,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
269,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
270,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
X.columns = [x + 48 * y for x in range(0, 48) for y in range(0, 10)]
X.head()

,0,48,96,144,192,240,288,336,384,432,...,47,95,143,191,239,287,335,383,431,479
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
X['DC'] = DC
X['Obj'] = Obj
X.head()
X.tail()

,0,48,96,144,192,240,288,336,384,432,...,143,191,239,287,335,383,431,479,DC,Obj
266,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,104.0,20.0
267,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,103.0,20.0
268,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,102.0,20.0
269,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,101.0,20.0
270,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,100.0,20.0


In [8]:
pointsData = pd.read_csv('PointsData.csv')
pointsData = pointsData.iloc[:,1:3]
pointsData.tail()

,X,Y
475,675,510
476,675,544
477,675,578
478,675,612
479,675,646


In [81]:
# for every row,
#   oneEntries <- python list (vector) of column indexes where entry == 1  (length should == Obj)
#   Subset the ``points'' dataframe to get X and Y values for those points
#   X, Y, DC, Obj, # Radars (length of oneEntries) gets row binded onto new dataframe
#      X.loc[:, 'Obj'].sum() shows should be 1698 points -> num rows in new dataframe
#   pd.concat:   axis = 0 is rbind, axis = 1 is cbind

allData = pd.DataFrame()
for rowIndex in range(X.shape[0]):
    oneEntries = (X.iloc[rowIndex,:] == 1)
    oneEntries = oneEntries[oneEntries].index
    dc = X.iloc[rowIndex, 480]
    obj = X.iloc[rowIndex, 481]
    DcObj = pd.concat([pd.Series(np.repeat(dc, obj)), pd.Series(np.repeat(obj, obj))], axis = 1, ignore_index = True)
    data = pd.concat([pointsData.iloc[oneEntries,:].reset_index(drop=True), DcObj], 
                     axis = 1, ignore_index = True)
    data.columns = ['X', 'Y', 'DC', '# of Radars']
    allData = pd.concat([allData, data], axis = 0)
    allData.reset_index(drop=True, inplace = True)
    
print(allData.shape)
print(allData.head())
allData.tail()
    

(1698, 4)
     X    Y     DC  # of Radars
0  297  204  370.0          2.0
1  351  544  370.0          2.0
2  297  204  369.0          2.0
3  351  544  369.0          2.0
4  297  204  368.0          2.0


,X,Y,DC,# of Radars
1693,243,34,100.0,20.0
1694,54,238,100.0,20.0
1695,108,544,100.0,20.0
1696,243,204,100.0,20.0
1697,378,272,100.0,20.0


In [100]:
# Testing one case

#print(allData.iloc[100:103,:])
#X.iloc[35,480:]
#oneEntries = (X.iloc[35,:] == 1)
#print(oneEntries)
#oneEntries = oneEntries[oneEntries].index
#print(oneEntries)
#print(pointsData.iloc[144,:])
#print(pointsData.iloc[395,:])
#print(pointsData.iloc[161,:])

       X    Y     DC  # of Radars
100  189  442  335.0          3.0
101  540  340  335.0          3.0
102  216  272  335.0          3.0
X    189
Y    442
Name: 144, dtype: int64
X    540
Y    340
Name: 395, dtype: int64
X    216
Y    272
Name: 161, dtype: int64


In [2]:
Longs = np.arange(-114.5, -101.5, 0.5)
Lats = np.concatenate((np.array([31.3]), np.array(np.arange(32, 42.5, 0.5))))
print(Longs)
print(Lats)

[-114.5 -114.  -113.5 -113.  -112.5 -112.  -111.5 -111.  -110.5 -110.
 -109.5 -109.  -108.5 -108.  -107.5 -107.  -106.5 -106.  -105.5 -105.
 -104.5 -104.  -103.5 -103.  -102.5 -102. ]
[31.3 32.  32.5 33.  33.5 34.  34.5 35.  35.5 36.  36.5 37.  37.5 38.
 38.5 39.  39.5 40.  40.5 41.  41.5 42. ]


In [102]:
allData['Longitude'] = [Longs[int(x / 27)] for x in allData.loc[:, 'X']]
allData['Latitude'] = [Lats[int(x / 34)] for x in allData.loc[:, 'Y']]
print(allData.head())
allData.tail()

     X    Y     DC  # of Radars  Longitude  Latitude
0  297  204  370.0          2.0      109.0      34.5
1  351  544  370.0          2.0      108.0      39.5
2  297  204  369.0          2.0      109.0      34.5
3  351  544  369.0          2.0      108.0      39.5
4  297  204  368.0          2.0      109.0      34.5


,X,Y,DC,# of Radars,Longitude,Latitude
1693,243,34,100.0,20.0,110.0,32.0
1694,54,238,100.0,20.0,113.5,35.0
1695,108,544,100.0,20.0,112.5,39.5
1696,243,204,100.0,20.0,110.0,34.5
1697,378,272,100.0,20.0,107.5,35.5


In [103]:
allData.to_csv('AllData.csv')

In [60]:
# test case for above

#print(np.where(np.array(X.iloc[4,:]) == 1))
#df.columns[(df == 38.15).iloc[0]]
oneEntries = (X.iloc[4,:] == 1)
oneEntries = oneEntries[oneEntries].index
print(type(oneEntries))
print(oneEntries)
#print(X.iloc[4,270:290:])
print(pointsData.iloc[oneEntries,:])
print(X.iloc[4, 480:])
a = pd.concat([pd.DataFrame(), pointsData.iloc[oneEntries,:]], axis = 0)
a.columns = ['A', 'B']
a

<class 'pandas.core.indexes.base.Index'>
Index([219, 268], dtype='object')
       X    Y
219  297  204
268  351  544
DC     366.0
Obj      2.0
Name: 4, dtype: float64


,A,B
219,297,204
268,351,544


In [47]:
# shoot our columns are messed up...
#[x + 48 * y for x in range(0, 48) for y in range(0, 10)]

[0,
 48,
 96,
 144,
 192,
 240,
 288,
 336,
 384,
 432,
 1,
 49,
 97,
 145,
 193,
 241,
 289,
 337,
 385,
 433,
 2,
 50,
 98,
 146,
 194,
 242,
 290,
 338,
 386,
 434,
 3,
 51,
 99,
 147,
 195,
 243,
 291,
 339,
 387,
 435,
 4,
 52,
 100,
 148,
 196,
 244,
 292,
 340,
 388,
 436,
 5,
 53,
 101,
 149,
 197,
 245,
 293,
 341,
 389,
 437,
 6,
 54,
 102,
 150,
 198,
 246,
 294,
 342,
 390,
 438,
 7,
 55,
 103,
 151,
 199,
 247,
 295,
 343,
 391,
 439,
 8,
 56,
 104,
 152,
 200,
 248,
 296,
 344,
 392,
 440,
 9,
 57,
 105,
 153,
 201,
 249,
 297,
 345,
 393,
 441,
 10,
 58,
 106,
 154,
 202,
 250,
 298,
 346,
 394,
 442,
 11,
 59,
 107,
 155,
 203,
 251,
 299,
 347,
 395,
 443,
 12,
 60,
 108,
 156,
 204,
 252,
 300,
 348,
 396,
 444,
 13,
 61,
 109,
 157,
 205,
 253,
 301,
 349,
 397,
 445,
 14,
 62,
 110,
 158,
 206,
 254,
 302,
 350,
 398,
 446,
 15,
 63,
 111,
 159,
 207,
 255,
 303,
 351,
 399,
 447,
 16,
 64,
 112,
 160,
 208,
 256,
 304,
 352,
 400,
 448,
 17,
 65,
 113,
 161,
 209,
